### Setup

In [2]:
import json
import time
from functools import reduce

import pandas as pd
import numpy as np

from ast import literal_eval

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
from tqdm.autonotebook import tqdm

# import seaborn as sns
# sns.set(rc={'figure.figsize':(12,8)})
# sns.set(style='whitegrid')

# import matplotlib.pyplot as plt

from py2neo import Graph, Node, Relationship

In [3]:
# public_address = '54.174.175.98'
# public_address = '54.88.167.164'
public_address = 'top42_neo4j'

graph = Graph('bolt://{}:7687'.format(public_address), auth=('neo4j','myneo'))

def run_query(query, graph, print_query=False, run_query=True, 
              print_only=False, to_df=False, verbose=True):
    df = 1
    if print_only: 
        print_query = True
        run_query = False
    start_time = time.time()
    if print_query:
        print(query)
    if run_query:
        if to_df:
            df = graph.run(query).to_data_frame()
        else:
            graph.run(query)
    end_time = time.time()
    minutes_elapsed = (end_time-start_time)/60
    if verbose:
        print("Query completed in {:.2f} minutes.".format(minutes_elapsed))
    return df


In [4]:
years_to_consider = range(1950, 2019)
max_years_to_predict = 10
years_tracked = 5

vars_to_use = ['adopters', 'timeScaledPageRank', 'citations', 'node2vec']

max_year = max(years_to_consider)
min_year = min(years_to_consider)

### Write data to CSV

In [38]:
metrics_pattern = 'MATCH (q)-[m{y}:METRICS_IN]->(:Year {{year:y.year+{y}}})'
metrics_string_list = [metrics_pattern.format(y=i) for i in range(years_tracked+1)]
metrics_string = '\n'.join(metrics_string_list)

var_pattern = 'coalesce(m{y}.{v},0) AS {v}_{y},'
var_string_list = [var_pattern.format(y=i,v=j) for i in range(years_tracked+1) for j in vars_to_use]
var_string = '\n'.join(var_string_list)

where_pattern = 'exists(m{y}.node2vec)'
where_string_list = [where_pattern.format(y=i) for i in range(2)]#years_tracked+1)]
where_string = ' AND '.join(where_string_list)       

author_metrics_pattern = 'MATCH (q)-[ma{y}:METRICS_IN]->(:Year {{year:y.year+{y}}})'
author_metrics_string_list = [author_metrics_pattern.format(y=i) for i in range(years_tracked+1)]
author_metrics_string = '\n'.join(author_metrics_string_list)

author_vars = ['total_papers', 'author_age', 'max_citations', 'num_venues', 'total_citations']
author_funs = ['max', 'sum']
author_var_pattern = '{f}(coalesce(ma{y}.{v},0)) AS {v}_{f}_{y},'
author_var_string_list = [author_var_pattern.format(y=i, v=j, f=k)
                          for i in range(years_tracked+1)
                          for j in author_vars
                          for k in author_funs]
author_var_string = '\n'.join(author_var_string_list)


query = """
MATCH (a:Author)-[:AUTHORED]->(q:Quanta)-[:PUBLISHED_IN]->(y:Year)
WHERE y.year>={miny} AND y.year<={maxy}//-{ya}
{metrics_string}
{author_metrics_string}
WHERE {where_string}
RETURN 
    {var_string}
    {author_var_string}
    id(q) AS id, 
    y.year AS year
""".format(miny=min_year, 
           maxy=max_year,
           ya=max_years_to_predict,
           metrics_string=metrics_string,
           var_string=var_string, 
           where_string=where_string, 
           author_metrics_string='',
           author_var_string='')

query_tocsv = """
CALL apoc.export.csv.query('
{q}
','/import/quanta.predict.{miny}.{maxy}.{yt}.csv', 
{{quotes:true}});
""".format(q=query, 
           miny=min_year, 
           maxy=max_year,
           yt=years_tracked)

run_query(query_tocsv, graph, to_df=False, print_only=True)


CALL apoc.export.csv.query('

MATCH (a:Author)-[:AUTHORED]->(q:Quanta)-[:PUBLISHED_IN]->(y:Year)
WHERE y.year>=1950 AND y.year<=2018//-10
MATCH (q)-[m0:METRICS_IN]->(:Year {year:y.year+0})
MATCH (q)-[m1:METRICS_IN]->(:Year {year:y.year+1})
MATCH (q)-[m2:METRICS_IN]->(:Year {year:y.year+2})
MATCH (q)-[m3:METRICS_IN]->(:Year {year:y.year+3})
MATCH (q)-[m4:METRICS_IN]->(:Year {year:y.year+4})
MATCH (q)-[m5:METRICS_IN]->(:Year {year:y.year+5})

WHERE exists(m0.node2vec) AND exists(m1.node2vec)
RETURN 
    coalesce(m0.adopters,0) AS adopters_0,
coalesce(m0.timeScaledPageRank,0) AS timeScaledPageRank_0,
coalesce(m0.citations,0) AS citations_0,
coalesce(m0.node2vec,0) AS node2vec_0,
coalesce(m1.adopters,0) AS adopters_1,
coalesce(m1.timeScaledPageRank,0) AS timeScaledPageRank_1,
coalesce(m1.citations,0) AS citations_1,
coalesce(m1.node2vec,0) AS node2vec_1,
coalesce(m2.adopters,0) AS adopters_2,
coalesce(m2.timeScaledPageRank,0) AS timeScaledPageRank_2,
coalesce(m2.citations,0) AS citations_

1

### Write Data To CSV (With Author Features)

In [ ]:
query_ex = """
MATCH (q)-[:PUBLISHED_IN]->(y:year {{year: {}}}) 
MATCH (q)-[m:METRICS_IN]->(y)
MATCH (a)-[:AUTHORED]-(q)
MATCH ()

"""


metrics_pattern = 'MATCH (q)-[m{y}:METRICS_IN]->(:Year {{year:y.year+{y}}})'
metrics_string_list = [metrics_pattern.format(y=i) for i in range(years_tracked+1)]
metrics_string = '\n'.join(metrics_string_list)

var_pattern = 'coalesce(m{y}.{v},0) AS {v}_{y},'
var_string_list = [var_pattern.format(y=i,v=j) for i in range(years_tracked+1) for j in vars_to_use]
var_string = '\n'.join(var_string_list)

where_pattern = 'exists(m{y}.node2vec)'
where_string_list = [where_pattern.format(y=i) for i in range(years_tracked+1)]
where_string = ' AND '.join(where_string_list)       

author_metrics_pattern = 'MATCH (q)<-[:AUTHORED]-(a:Author)-[ma{y}:METRICS_IN]->(:Year {{year:y.year+{y}}})' #changed from q to a
author_metrics_string_list = [author_metrics_pattern.format(y=i) for i in range(years_tracked+1)]
author_metrics_string = '\n'.join(author_metrics_string_list)

# node2vec with different nodes types
# change only to average
#author_vars = ['total_papers', 'author_age', 'max_citations', 'num_venues', 'total_citations']
author_vars = [
    'hIndex', 'hIndexDelta', 'totalCitations', 'totalCitationsDelta', 'citationsPerPaper', 
    'citationsPerPaperDelta','citationsPerYear', 'totalPapers', 'totalPapersDelta', 'rankCitationsPerYear',
    'pageRank', 'weightedPageRank', 'authorAge', 'recentCoauthors', 'maxCitations', 'totalVenues', 
    'venueHIndexMean', 'venueHIndexDeltaMean', 'venueCitationsPerPaperMean', 'venueCitationsPerPaperDeltaMean',  
    'venueTotalPapersMean', 'venueTotalPapersDeltaMean', 'venueRankCitationsPerPaperMean', 'venueMaxCitationsMean']

author_funs = ['avg', 'max', 'sum'] # why are we doing sum here? should we do average instead or in addition to sum?
author_var_patterns = 
author_var_pattern = '{f}(coalesce(ma{y}.{v},0)) AS {v}_{f}_{y},'
author_var_string_list = [author_var_pattern.format(y=i, v=j, f=k)
                          for i in range(years_tracked+1)
                          for j in author_vars
                          for k in author_funs]
author_var_string = '\n'.join(author_var_string_list)

# Do we need any with statements
query = """
MATCH (a:Author)-[:AUTHORED]->(q:Quanta)-[:PUBLISHED_IN]->(y:Year)
WHERE y.year>={miny} AND y.year<={maxy}//-{ya}
{metrics_string}
{author_metrics_string}
WHERE {where_string}
RETURN 
    {var_string}
    {author_var_string}
    id(q) AS id, 
    y.year AS year
""".format(miny=min_year, 
           maxy=max_year,
           ya=max_years_to_predict,
           metrics_string=metrics_string,
           var_string=var_string, 
           where_string=where_string, 
           author_metrics_string='',
           author_var_string='')

query_tocsv = """
CALL apoc.export.csv.query('
{q}
','/import/quanta.predict.{miny}.{maxy}.{yt}.csv', 
{{quotes:true}});
""".format(q=query, 
           miny=min_year, 
           maxy=max_year,
           yt=years_tracked)

run_query(query_tocsv, graph, to_df=False, print_only=True)

### Read in data for each year, format it, and write it back

In [ ]:
fpath = '/tmp/data/quanta.predict.{miny}.{maxy}.{yt}.csv'.format(
    miny=min_year, maxy=max_year, yt=years_tracked)
df = pd.read_csv(fpath, engine='python', error_bad_lines=False)
df = df.dropna()

for y in tqdm(range(years_tracked+1)):

    col = 'node2vec_{}'.format(y)    
    n2vdf = pd.DataFrame(df[col].apply(json.loads).tolist())
    n2v_dim = n2vdf.shape[1]
    n2vdf.columns = ['{}_{}'.format(col, i) for i in range(n2v_dim)]

    df = pd.concat([df.reset_index(drop=True), n2vdf.reset_index(drop=True)], axis=1)
    df = df.drop(col, axis=1)


df.to_csv('{}.out'.format(fpath))

### Read in prediction data and make predictions

In [18]:
from sklearn.model_selection import train_test_split, validation_curve
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler, QuantileTransformer, RobustScaler
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier
from sklearn.metrics import f1_score, mean_squared_error, classification_report, balanced_accuracy_score
from sklearn.linear_model import LogisticRegression

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.pipeline import make_pipeline, Pipeline

from sklearn.externals.joblib import parallel_backend


%env JOBLIB_TEMP_FOLDER=/tmp

env: JOBLIB_TEMP_FOLDER=/tmp


/home/jovyan/.local/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/home/jovyan/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [23]:
df = pd.read_csv('/tmp/data/quanta.predict.{miny}.{maxy}.{yt}.csv.out'.format(
    miny=min_year, maxy=max_year, yt=years_tracked))

In [24]:
df.describe()

,Unnamed: 0,adopters_0,adopters_1,adopters_2,adopters_3,adopters_4,adopters_5,citations_0,citations_1,citations_2,...,node2vec_5_2,node2vec_5_3,node2vec_5_4,node2vec_5_5,node2vec_5_6,node2vec_5_7,node2vec_5_8,node2vec_5_9,node2vec_5_10,node2vec_5_11
count,5.262588e+06,5.262588e+06,5.262588e+06,5.262588e+06,5.262588e+06,5.262588e+06,5.262588e+06,5.262588e+06,5.262588e+06,5.262588e+06,...,5.262588e+06,5.262588e+06,5.262588e+06,5.262588e+06,5.262588e+06,5.262588e+06,5.262588e+06,5.262588e+06,5.262588e+06,5.262588e+06
mean,2.631294e+06,1.132223e+01,2.889139e+01,3.120092e+01,2.497449e+01,5.485345e-01,7.468759e-02,8.941316e-01,2.060872e+00,2.016791e+00,...,7.420117e+12,9.878786e+12,7.043323e+13,-5.330409e+13,-1.397789e+13,1.691927e+12,2.406043e+13,3.138254e+13,9.032189e+13,-1.366777e+14
std,1.519178e+06,6.883398e+01,1.196610e+02,1.421127e+02,1.193722e+02,3.936234e+01,5.916535e+00,2.479747e+00,4.744520e+00,4.934994e+00,...,1.228760e+14,1.632540e+14,7.140607e+14,5.624136e+14,2.831408e+14,5.939874e+13,5.096146e+14,7.109472e+14,7.441738e+14,1.055877e+15
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-4.462150e+15,-5.927330e+15,-1.654170e+16,-3.072070e+16,-1.475570e+16,-7.837340e+15,-1.293990e+16,-1.804400e+16,-1.705340e+16,-6.349120e+16
25%,1.315647e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-1.173800e-01,-3.903130e-01,-2.209800e-01,-8.322860e-01,-8.384342e-01,-4.416950e-01,-3.936530e-01,-1.720010e-01,-5.172780e-01,-7.182760e-01
50%,2.631294e+06,0.000000e+00,3.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,...,3.850575e-02,-2.689320e-02,8.682170e-03,-5.328800e-02,-3.869480e-02,-3.606940e-02,-2.413070e-02,7.077720e-02,4.356530e-02,-9.097480e-02
75%,3.946940e+06,4.000000e+00,2.000000e+01,2.100000e+01,1.600000e+01,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,...,5.496510e-01,1.934150e-01,2.613402e-01,1.444350e-01,1.796960e-01,1.637490e-01,4.282600e-01,5.274802e-01,4.511310e-01,3.999810e-01
max,5.262587e+06,2.992000e+03,5.025000e+03,5.025000e+03,4.999000e+03,4.015000e+03,7.730000e+02,9.900000e+01,1.800000e+02,2.050000e+02,...,5.361910e+15,7.120870e+15,3.887430e+16,1.355800e+16,3.369140e+16,3.422790e+15,5.631000e+16,7.780440e+16,4.669420e+16,2.350950e+16


In [25]:
df.index[df['year'] <= max_year - years_tracked + 1]

Int64Index([      0,       1,       2,       3,       4,       5,       6,
                  7,       8,       9,
            ...
            5262578, 5262579, 5262580, 5262581, 5262582, 5262583, 5262584,
            5262585, 5262586, 5262587],
           dtype='int64', length=5262588)

In [ ]:


results = []

for years_to_track in tqdm(range(years_tracked)):
#     for year_to_predict in range(years_to_track+1, max_years_to_predict+1):
#         print(years_to_track, year_to_predict)
#         pass
    year_to_predict = years_to_track + 1
    
    cols_to_keep = ['{v}_{y}'.format(y=i,v=j) 
        for i in range(years_to_track+1) 
        for j in [v for v in vars_to_use if v!='node2vec']]

    n2v_cols_to_keep =  ['node2vec_{y}_{i}'.format(y=y, i=i) 
                     for y in range(years_to_track+1)
                     for i in range(n2v_dim)]
        
    cols_to_keep = cols_to_keep + n2v_cols_to_keep
    X = df.loc[:, cols_to_keep]
    
    y_col = 'timeScaledPageRank_{y}'.format(y=year_to_predict)
    y = df.loc[:, y_col] > df[y_col].quantile(q=.95)


    pipeline = Pipeline(
        memory=None,
        steps=[
            ('spl', SMOTE()),
            ('scl', QuantileTransformer()),
            ('clf', RandomForestClassifier())
        ]
    )

    grid = {'clf__n_estimators': [int(x) for x in np.linspace(200, 2000, num=10)],
            'clf__max_features': ['auto', 'sqrt'],
            'clf__max_depth': [int(x) for x in np.linspace(10, 1000, num=10)],
            'clf__min_samples_split': [2, 5, 10],
            'clf__min_samples_leaf': [1, 2, 4],
            'clf__bootstrap': [True, False]}

    random_search = RandomizedSearchCV(
        estimator=pipeline, 
        param_distributions=grid, 
        n_iter=10, 
        cv=3, 
        n_jobs=-1,
        verbose=3,
        random_state=42
    )


    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    with parallel_backend('threading'):
        random_search.fit(X_train, y_train)

    y_pred = random_search.predict(X_test)

    results.append({
        'years_tracked': years_to_track, 
        'year_predicted': year_to_predict,
        'score': random_search.score(X=X_test, y=y_test),
        'f1': f1_score(y_pred=y_pred, y_true=y_test),
        'balanced_accuracy': balanced_accuracy_score(y_true=y_test, y_pred=y_pred),
        'balanced_accuracy_adjusted': balanced_accuracy_score(y_true=y_test, y_pred=y_pred, adjusted=True),
        'classification_report': classification_report(y_true=y_test, y_pred=y_pred, output_dict=True),
        'random_search': random_search
    })

    pickle.dump(results, open('prediction_models_{}.pickle'.format(years_to_track),'wb'))
    
pickle.dump(results, open('prediction_models.pickle','wb'))

In [ ]:
years_to_track=3
year_to_predict=5
n2v_dim=12

results = []

cols_to_keep = ['{v}_{y}'.format(y=i,v=j) 
        for i in range(years_to_track+1) 
        for j in [v for v in vars_to_use if v!='node2vec']]

n2v_cols_to_keep =  ['node2vec_{y}_{i}'.format(y=y, i=i) 
                 for y in range(years_to_track+1)
                 for i in range(n2v_dim)]

cols_to_keep = cols_to_keep + n2v_cols_to_keep
X = df.loc[:, cols_to_keep]

y_col = 'timeScaledPageRank_{y}'.format(y=year_to_predict)
y = df.loc[:, y_col] > df[y_col].quantile(q=.95)


pipeline = Pipeline(
    memory=None,
    steps=[
        ('spl', RandomUnderSampler()),
        ('scl', QuantileTransformer()),
        ('clf', RandomForestClassifier())
    ]
)

grid = {'clf__n_estimators': [int(x) for x in np.linspace(200, 2000, num=10)],
        'clf__max_features': ['auto', 'sqrt'],
        'clf__max_depth': [int(x) for x in np.linspace(10, 1000, num=10)],
        'clf__min_samples_split': [2, 5, 10],
        'clf__min_samples_leaf': [1, 2, 4],
        'clf__bootstrap': [True, False]}

random_search = RandomizedSearchCV(
    estimator=pipeline, 
    param_distributions=grid, 
    n_iter=15, 
    cv=3, 
    n_jobs=-1,
    verbose=3,
    random_state=42
)


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

with parallel_backend('threading'):
    random_search.fit(X_train, y_train)
    
y_pred = random_search.predict(X_test)

results.append({
    'years_tracked': years_to_track, 
    'year_predicted': year_to_predict,
    'score': random_search.score(X=X_test, y=y_test),
    'f1': f1_score(y_pred=y_pred, y_true=y_test),
    'balanced_accuracy': balanced_accuracy_score(y_true=y_test, y_pred=y_pred),
    'balanced_accuracy_adjusted': balanced_accuracy_score(y_true=y_test, y_pred=y_pred, adjusted=True),
    'classification_report': classification_report(y_true=y_test, y_pred=y_pred, output_dict=True),
    'random_search': random_search
})

print(classification_report(y_true=y_test, y_pred=y_pred))

In [ ]:
results


In [ ]:
print(classification_report(y_true=y_test, y_pred=y_pred))

In [ ]:
pipeline.get_params()


In [ ]:
years_to_track=1
year_to_predict=10
n2v_dim=12

results = pd.DataFrame(columns=['years_tracked', 'year_predicted', 'score'])


vars_to_use = ['adopters', 'citatons', 'node2vec']

cols_to_keep = ['{v}_{y}'.format(y=i,v=j) 
        for i in range(years_to_track+1) 
        for j in [v for v in vars_to_use if v!='node2vec']]

n2v_cols_to_keep =  ['node2vec_{y}_{i}'.format(y=y, i=i) 
                 for y in range(years_to_track+1)
                 for i in range(n2v_dim)]

cols_to_keep = cols_to_keep + n2v_cols_to_keep
X = df.loc[:, cols_to_keep]

y_col = ['timeScaledPageRank_{y}'.format(y=i) for i in range(2,10)]


y = df.loc[:, y_col] > df[y_col].quantile(q=.9)


param_grid = {
    "max_depth": [10, None],
    "max_features": [5, 10, 20],
    "n_estimators": [10, 100, 200]
}

# pipeline = make_pipeline(
# #     SMOTE(),
#     RandomUnderSampler(),
# #     QuantileTransformer(),
#     MinMaxScaler(),
#     GridSearchCV(RandomForestClassifier(class_weight={True:3, False:1}),
#                  param_grid=param_grid, 
#                  refit=True, 
#                  n_jobs=-1, 
#                  cv=2, 
#                  scoring='f1') 
# )


pipeline = Pipeline(
    memory=None,
    steps=[
#         ('spl', SMOTE()),
#         ('scl', MinMaxScaler()),
        ('clf', RandomForestClassifier())
    ]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

results = results.append({
    'years_tracked': years_to_track, 
    'year_predicted': year_to_predict,
    'score': pipeline.score(X=X_test, y=y_test),
#     'f1': f1_score(y_pred=y_pred, y_true=y_test)
}, ignore_index=True)

print(classification_report(y_true=y_test, y_pred=y_pred))

In [ ]:
pipeline.predict_proba(df.loc[df['id'] == 29052283, cols_to_keep])

In [ ]:
from plot_metric.functions import  BinaryClassification

bc = BinaryClassification(y_test, y_prob, labels=["Low Impact", "High Impact"])

plt.figure(figsize=(15,10))

plt.subplot2grid(shape=(2,6), loc=(0,0), colspan=2)
bc.plot_roc_curve()

plt.subplot2grid((2,6), (0,2), colspan=2)
bc.plot_precision_recall_curve()

plt.subplot2grid((2,6), (0,4), colspan=2)
bc.plot_class_distribution()

plt.subplot2grid((2,6), (1,1), colspan=2)
bc.plot_confusion_matrix()

plt.subplot2grid((2,6), (1,3), colspan=2)
bc.plot_confusion_matrix(normalize=True)

plt.show()
bc.print_report()

In [ ]:
y_pred = pipeline.predict(X_test)
f1_score(y_pred=y_pred, y_true=y_test)

In [ ]:
sns.heatmap(data=results.pivot('years_tracked', 'year_predicted', 'score'), 
           annot=True, fmt='.2f', linewidth=.5, cbar=True, square=True, 
           cmap='YlGnBu', center=results['score'].mean())

In [ ]:
plt.figure(figsize=(10,20))
feature_importance = pd.DataFrame.from_dict(
    dict(zip(X.columns,pipeline.steps[1][1].feature_importances_)), orient='index').T
sns.barplot(orient='h',data=feature_importance)

In [ ]:
pipeline

In [ ]:
.shape

In [ ]:
from sklearn.model_selection import validation_curve

param = 'randomforestclassifier__max_depth'
param_range = list(range(1,100,25))
n_cv = 2

train_scores, valid_scores = validation_curve(pipeline, 
                                              X=X, 
                                              y=y,
                                              n_jobs=-1,
                                              param_name=param,
                                              scoring='roc_auc',
                                              param_range=param_range,
                                              cv=n_cv)

vdf = pd.DataFrame(np.concatenate([train_scores, valid_scores]),
             columns=['cv_fold_{}'.format(i) for i in range(n_cv)],
            )
vdf[param] = param_range*2
vdf['type'] = ['train']*len(param_range) + ['valid']*len(param_range)

sns.lineplot(data=vdf.melt(id_vars=['type', param]), 
             x=param, y='value', hue='type')

In [ ]:
pd.concat([X_train, X_test]).head()

In [ ]:
X.columns == 

### Predictions

In [ ]:
from tpot import TPOTRegressor
tpot = TPOTRegressor(generations=5, population_size=20, verbosity=2, n_jobs=-1, 
                                                    scoring='r2', config_dict='TPOT light', 
                                                    max_time_mins=30, max_eval_time_mins=5) 
tpot.fit(X_train, y_train)

In [ ]:
tpot.score(X_test, y_test)

In [ ]:
tpot.export('tpot_exported_pipeline.py')

In [ ]:
!cat tpot_exported_pipeline.py

In [ ]:
years_ahead = 4
df = pd.read_csv('/tmp/data/quanta.predict.{miny}.{maxy}.{ya}.csv.out'.format(
        miny=min_year, maxy=max_year, ya=years_ahead))
df.shape

In [ ]:
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *